In [89]:
from tqdm.notebook import tqdm
import numpy as np
import sys
sys.path.append('../')
from partitioning import is_pow_of_two, init_membership_list
# from spawn import start_db_background
from structures import Params, NetworkParams
from client_dynamo import client_get, client_put, client_get_memory, client_fail
from parallel_runner import run_parallel
from random import randint

# import logging
# logger = logging.getLogger('dynamo_node')
# logger.setLevel(logging.DEBUG)

START_PORT = 2333
CLIENT_ID = 1

params = Params({
    'num_proc' : 8,
    'hash_size': 8, # 2^3 = 8 
    'Q' : 16, # 
    'N' : 4,
    'w_timeout': 2,
    'r_timeout': 2,
    'R': 1,
    'W': 1,
    'gossip': True,
    'update_failure_on_rpcs': False,
})

network_params = NetworkParams({
    'latency': 50,
    'randomize_latency': True,
    'drop_prob': 0
})

def get_start_port(randomize=True):
    return START_PORT + randint(0, params.num_proc-1) * int(randomize)

In [206]:
import time

init_val = "1"
delta = 50 # in ms

In [225]:
# Initial value
port = get_start_port()
print(f'Sending request to port {port}')
response = client_put(port, CLIENT_ID, key=1, val=init_val)
context = response.context

# New value
new_val = f'{init_val}1'
port = get_start_port()
print(f'Sending request to port {port}')
client_put(port, CLIENT_ID, key=1, val=new_val, context=context)
start = time.time()

# Send get after delay
time.sleep(delta / 1000)
port = get_start_port()
print(f'Sending request to port {port}')
response = client_get(port, CLIENT_ID, key=1)
init_val = new_val

Sending request to port 2336
Sending request to port 2337
Sending request to port 2336


In [226]:
if response.items[0].val == new_val:
    print("Consistent")
else:
    print("Inconsistent")

Inconsistent


In [233]:
from tqdm.notebook import tqdm

def check_consistent(old_val, delta, key=1):
    # New value
    new_val = f'{old_val}1'
    port = get_start_port()
    client_put(port, CLIENT_ID, key=key, val=new_val, context=context)
    start = time.time()

    # Send get after delay
    time.sleep(delta / 1000)
    port = get_start_port()
    response = client_get(port, CLIENT_ID, key=1)
    if response.items[0].val == new_val:
        return True, new_val
    else:
        return False, new_val

def check_consistent_multiple(num_requests=100, delta=0, key=1):
    old_val = "1"
    client_put(get_start_port(), CLIENT_ID, key=key, val=old_val)
    
    correct = 0
    for i in tqdm(range(num_requests)):
        consistent, old_val = check_consistent(old_val, delta, key)
        if consistent:
            correct += 1

    return correct * 100 / num_requests

In [234]:
check_consistent_multiple(1000, 0)

64.2

In [ ]:
check_consistent_multiple(1000, 5)

In [238]:
#NRW = 4,1,1, normal

deltas = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60, 70, 80, 90, 100]
num_requests = 1000

percentages = []

for delta in deltas:
    print(f"Running {num_requests} requests with delta = {delta}")
    result = check_consistent_multiple(num_requests, delta)
    percentages.append(result)

Running 1000 requests with delta = 0



Running 1000 requests with delta = 5



Running 1000 requests with delta = 10



Running 1000 requests with delta = 15



Running 1000 requests with delta = 20


...


Running 1000 requests with delta = 60



Running 1000 requests with delta = 70



Running 1000 requests with delta = 80



Running 1000 requests with delta = 90



Running 1000 requests with delta = 100


In [239]:
percentages

[75.3,
 79.0,
 80.8,
 83.9,
 85.2,
 85.8,
 84.8,
 88.6,
 86.7,
 91.4,
 91.0,
 93.6,
 95.7,
 94.9,
 97.4,
 98.0]

In [240]:
#NRW = 4,1,1, uniform

deltas = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60, 70, 80, 90, 100]
num_requests = 1000

percentages = []

for delta in deltas:
    print(f"Running {num_requests} requests with delta = {delta}")
    result = check_consistent_multiple(num_requests, delta)
    print(f"Result: {result}")
    percentages.append(result)

Running 1000 requests with delta = 0



Result: 79.1
Running 1000 requests with delta = 5



Result: 81.1
Running 1000 requests with delta = 10



Result: 83.9
Running 1000 requests with delta = 15



Result: 85.6
Running 1000 requests with delta = 20


...


Result: 98.7
Running 1000 requests with delta = 70



Result: 98.6
Running 1000 requests with delta = 80



Result: 99.9
Running 1000 requests with delta = 90



Result: 99.9
Running 1000 requests with delta = 100



Result: 100.0


In [241]:
percentages

[79.1,
 81.1,
 83.9,
 85.6,
 87.2,
 89.9,
 92.5,
 92.6,
 95.0,
 95.3,
 96.8,
 98.7,
 98.6,
 99.9,
 99.9,
 100.0]

In [242]:
#NRW = 4,1,2, normal

deltas = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60, 70, 80, 90, 100]
num_requests = 1000

percentages = []

for delta in deltas:
    print(f"Running {num_requests} requests with delta = {delta}")
    result = check_consistent_multiple(num_requests, delta)
    print(f"Result: {result}")
    percentages.append(result)

Running 1000 requests with delta = 0



Result: 84.4
Running 1000 requests with delta = 5



Result: 85.3
Running 1000 requests with delta = 10



Result: 87.8
Running 1000 requests with delta = 15



Result: 89.2
Running 1000 requests with delta = 20


...


Result: 96.6
Running 1000 requests with delta = 70



Result: 97.1
Running 1000 requests with delta = 80



Result: 97.9
Running 1000 requests with delta = 90



Result: 99.3
Running 1000 requests with delta = 100



Result: 99.1


In [243]:
percentages

[84.4,
 85.3,
 87.8,
 89.2,
 88.7,
 91.3,
 93.7,
 95.1,
 94.0,
 95.5,
 96.1,
 96.6,
 97.1,
 97.9,
 99.3,
 99.1]

In [244]:
#NRW = 4,1,2, uniform

deltas = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60, 70, 80, 90, 100]
num_requests = 1000

percentages = []

for delta in deltas:
    print(f"Running {num_requests} requests with delta = {delta}")
    result = check_consistent_multiple(num_requests, delta)
    print(f"Result: {result}")
    percentages.append(result)

Running 1000 requests with delta = 0



Result: 90.1
Running 1000 requests with delta = 5



Result: 91.5
Running 1000 requests with delta = 10



Result: 93.8
Running 1000 requests with delta = 15



Result: 95.2
Running 1000 requests with delta = 20


...


Result: 99.7
Running 1000 requests with delta = 70



Result: 99.7
Running 1000 requests with delta = 80



Result: 100.0
Running 1000 requests with delta = 90



Result: 100.0
Running 1000 requests with delta = 100



Result: 100.0


In [245]:
#NRW = 4,2,1, normal

deltas = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60, 70, 80, 90, 100]
num_requests = 1000

percentages = []

for delta in deltas:
    print(f"Running {num_requests} requests with delta = {delta}")
    result = check_consistent_multiple(num_requests, delta)
    print(f"Result: {result}")
    percentages.append(result)

Running 1000 requests with delta = 0



Result: 78.2
Running 1000 requests with delta = 5



Result: 82.0
Running 1000 requests with delta = 10



Result: 82.0
Running 1000 requests with delta = 15



Result: 86.7
Running 1000 requests with delta = 20


...


Result: 95.6
Running 1000 requests with delta = 70



Result: 95.6
Running 1000 requests with delta = 80



Result: 98.0
Running 1000 requests with delta = 90



Result: 97.5
Running 1000 requests with delta = 100



Result: 98.9


In [246]:
percentages

[78.2,
 82.0,
 82.0,
 86.7,
 87.4,
 87.0,
 90.0,
 89.6,
 88.2,
 93.8,
 92.3,
 95.6,
 95.6,
 98.0,
 97.5,
 98.9]

In [247]:
#NRW = 4,2,1, uniform

deltas = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60, 70, 80, 90, 100]
num_requests = 1000

percentages = []

for delta in deltas:
    print(f"Running {num_requests} requests with delta = {delta}")
    result = check_consistent_multiple(num_requests, delta)
    print(f"Result: {result}")
    percentages.append(result)

Running 1000 requests with delta = 0



Result: 86.7
Running 1000 requests with delta = 5



Result: 89.6
Running 1000 requests with delta = 10



Result: 93.3
Running 1000 requests with delta = 15



Result: 93.3
Running 1000 requests with delta = 20


...


Result: 99.2
Running 1000 requests with delta = 70



Result: 99.8
Running 1000 requests with delta = 80



Result: 99.9
Running 1000 requests with delta = 90



Result: 100.0
Running 1000 requests with delta = 100



Result: 100.0


In [248]:
percentages

[86.7,
 89.6,
 93.3,
 93.3,
 97.0,
 96.1,
 98.0,
 97.4,
 98.4,
 98.4,
 98.6,
 99.2,
 99.8,
 99.9,
 100.0,
 100.0]

In [249]:
#NRW = 4,2,3, normal

deltas = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60, 70, 80, 90, 100]
num_requests = 1000

percentages = []

for delta in deltas:
    print(f"Running {num_requests} requests with delta = {delta}")
    result = check_consistent_multiple(num_requests, delta)
    print(f"Result: {result}")
    percentages.append(result)

Running 1000 requests with delta = 0



Result: 95.1
Running 1000 requests with delta = 5



Result: 95.5
Running 1000 requests with delta = 10



Result: 97.3
Running 1000 requests with delta = 15



Result: 97.8
Running 1000 requests with delta = 20


...


Result: 99.6
Running 1000 requests with delta = 70



Result: 99.5
Running 1000 requests with delta = 80



Result: 99.4
Running 1000 requests with delta = 90



Result: 99.8
Running 1000 requests with delta = 100



Result: 100.0


In [250]:
percentages

[95.1,
 95.5,
 97.3,
 97.8,
 97.9,
 97.7,
 98.9,
 98.9,
 99.1,
 98.7,
 99.2,
 99.6,
 99.5,
 99.4,
 99.8,
 100.0]

In [251]:
#NRW = 4,2,3, uniform

deltas = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60, 70, 80, 90, 100]
num_requests = 1000

percentages = []

for delta in deltas:
    print(f"Running {num_requests} requests with delta = {delta}")
    result = check_consistent_multiple(num_requests, delta)
    print(f"Result: {result}")
    percentages.append(result)

Running 1000 requests with delta = 0



Result: 99.3
Running 1000 requests with delta = 5



Result: 99.6
Running 1000 requests with delta = 10



Result: 99.9
Running 1000 requests with delta = 15



Result: 99.5
Running 1000 requests with delta = 20


...


Result: 100.0
Running 1000 requests with delta = 70



Result: 100.0
Running 1000 requests with delta = 80



Result: 100.0
Running 1000 requests with delta = 90



Result: 100.0
Running 1000 requests with delta = 100



Result: 100.0


In [252]:
percentages

[99.3,
 99.6,
 99.9,
 99.5,
 99.9,
 99.9,
 99.9,
 99.9,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0]